In [1]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
#from selenium.webdriver.support.expected_conditions import element_selection_state_to_be
#import matplotlib.pyplot as plt
import base64
#import matplotlib.pyplot as plt
import io
import os
from math import floor

## web crawling packages
import time
from datetime import date
from datetime import datetime
import datetime
import requests
from lxml import html
import csv
import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
import selenium

## scrapping game id's from espn

In [3]:
//*[@id="401326436"]
//*[@id="401326436"]
/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section

SyntaxError: invalid syntax (<ipython-input-3-4a4b31a00c4a>, line 1)

In [4]:
/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[1]/div/section

NameError: name 'body' is not defined

In [20]:
driver_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(driver_path)
driver.get('https://www.espn.com/nfl/scoreboard/_/week/8/year/2021/seasontype/2')

In [21]:
test = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section')

test_list = []
for p in range(len(test)):
    test_list.append(test[p].get_attribute("id"))

test_list
print(len(test_list))

0


In [ ]:
/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[2]/div/section[1]

In [29]:
first = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[1]/div/section")

first_list = []
for p in range(len(first)):
    first_list.append(first[p].get_attribute("id"))

first_list

['401326436']

In [32]:
sunday = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[2]/div/section')

sunday_list = []
for p in range(len(sunday)):
    sunday_list.append(sunday[p].get_attribute("id"))

sunday_list

['401326437',
 '401326438',
 '401326439',
 '401326440',
 '401326441',
 '401326443',
 '401326444',
 '401326442',
 '401326445',
 '401326446',
 '401326447',
 '401326448',
 '401326449']

In [33]:
monday = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[3]/div/section')

monday_list = []
for p in range(len(monday)):
    monday_list.append(monday[p].get_attribute("id"))

monday_list

['401326450']

## starting code for the loop here

In [7]:
url_list = []

# year_list = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
year_list = ['2021', '2022']

for i in year_list:
    year = str(i)

    for i in range(1,19):
        week = str(i)
        url_list.append([year, week])

url_df = pd.DataFrame(url_list, columns = ['year', 'week'])
url_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + url_df['week'] + "/year/" + url_df['year'] + "/seasontype/2"
print(url_df.shape)
url_df.head()

(36, 3)


,year,week,url
0,2021,1,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,2021,2,https://www.espn.com/nfl/scoreboard/_/week/2/y...
2,2021,3,https://www.espn.com/nfl/scoreboard/_/week/3/y...
3,2021,4,https://www.espn.com/nfl/scoreboard/_/week/4/y...
4,2021,5,https://www.espn.com/nfl/scoreboard/_/week/5/y...


In [46]:
url_df = url_df[11:33]
url_df

,year,week,url
11,2021,12,https://www.espn.com/nfl/scoreboard/_/week/12/...
12,2021,13,https://www.espn.com/nfl/scoreboard/_/week/13/...
13,2021,14,https://www.espn.com/nfl/scoreboard/_/week/14/...
14,2021,15,https://www.espn.com/nfl/scoreboard/_/week/15/...
15,2021,16,https://www.espn.com/nfl/scoreboard/_/week/16/...
16,2021,17,https://www.espn.com/nfl/scoreboard/_/week/17/...
17,2021,18,https://www.espn.com/nfl/scoreboard/_/week/18/...
18,2022,1,https://www.espn.com/nfl/scoreboard/_/week/1/y...
19,2022,2,https://www.espn.com/nfl/scoreboard/_/week/2/y...
20,2022,3,https://www.espn.com/nfl/scoreboard/_/week/3/y...


In [10]:
# url_df_catch_up = url_df.iloc[351:353]
# url_df_catch_up

In [47]:
game_id_df = pd.DataFrame()

for i in url_df['url']:
    driver_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(driver_path)
    driver.get(i)

    ## temp dataframes as a holding place before appending all together at the end of the loop
    first_df_temp = pd.DataFrame()
    second_df_temp = pd.DataFrame()
    third_df_temp = pd.DataFrame()
    fourth_df_temp = pd.DataFrame()
    
    try:
        # first_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section')
        first_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[1]/div/section')

        first_list = []
        for p in range(len(first_day)):
            first_list.append(first_day[p].get_attribute("id"))
        first_df = pd.DataFrame(first_list, columns = ['id'])
        first_df_temp = first_df_temp.append(first_df)

    except Exception as e:
        print("Error Couldn't Find First element:", e)

    try:
        second_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[2]/div/section')

        second_list = []
        for p in range(len(second_day)):
            second_list.append(second_day[p].get_attribute("id"))
        second_df = pd.DataFrame(second_list, columns = ['id'])
        second_df_temp = second_df_temp.append(second_df)

    except Exception as e:
        print("Error Couldn't Find Second element:", e)

    try:
        third_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[3]/div/section')

        third_list = []
        for p in range(len(third_day)):
            third_list.append(third_day[p].get_attribute("id"))
        third_df = pd.DataFrame(third_list, columns = ['id'])
        third_df_temp = third_df_temp.append(third_df)

    except Exception as e:
        print("Error Couldn't Find Third element:", e)

    try:
        fourth_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[4]/div/section')

        fourth_list = []
        for p in range(len(fourth_day)):
            fourth_list.append(fourth_day[p].get_attribute("id"))
        fourth_df = pd.DataFrame(fourth_list, columns = ['id'])
        fourth_df_temp = fourth_df_temp.append(fourth_df)

    except Exception as e:
        print("Error Couldn't Find Fourth element:", e)

    game_id_df_temp = pd.DataFrame(first_df_temp.append([second_df_temp, third_df_temp, fourth_df_temp])).reset_index()
    game_id_df_temp['url'] = i
    game_id_df_temp = game_id_df_temp.drop(columns = ['index'])

    game_id_df = game_id_df.append(game_id_df_temp).reset_index()
    game_id_df = game_id_df.drop(columns = ['index'])

In [48]:
i = "https://www.espn.com/nfl/scoreboard/_/week/16/year/2022/seasontype/2"
driver.get(i)

## temp dataframes as a holding place before appending all together at the end of the loop
first_df_temp = pd.DataFrame()
second_df_temp = pd.DataFrame()
third_df_temp = pd.DataFrame()
fourth_df_temp = pd.DataFrame()

try:
    # first_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section')
    first_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[1]/div/section')

    first_list = []
    for p in range(len(first_day)):
        first_list.append(first_day[p].get_attribute("id"))
    first_df = pd.DataFrame(first_list, columns = ['id'])
    first_df_temp = first_df_temp.append(first_df)

except Exception as e:
    print("Error Couldn't Find First element:", e)

try:
    second_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[2]/div/section')

    second_list = []
    for p in range(len(second_day)):
        second_list.append(second_day[p].get_attribute("id"))
    second_df = pd.DataFrame(second_list, columns = ['id'])
    second_df_temp = second_df_temp.append(second_df)

except Exception as e:
    print("Error Couldn't Find Second element:", e)

try:
    third_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[3]/div/section')

    third_list = []
    for p in range(len(third_day)):
        third_list.append(third_day[p].get_attribute("id"))
    third_df = pd.DataFrame(third_list, columns = ['id'])
    third_df_temp = third_df_temp.append(third_df)

except Exception as e:
    print("Error Couldn't Find Third element:", e)

try:
    fourth_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/main/div[3]/div/div[1]/div[1]/div/section[4]/div/section')

    fourth_list = []
    for p in range(len(fourth_day)):
        fourth_list.append(fourth_day[p].get_attribute("id"))
    fourth_df = pd.DataFrame(fourth_list, columns = ['id'])
    fourth_df_temp = fourth_df_temp.append(fourth_df)

except Exception as e:
    print("Error Couldn't Find Fourth element:", e)

game_id_df_temp = pd.DataFrame(first_df_temp.append([second_df_temp, third_df_temp, fourth_df_temp])).reset_index()
game_id_df_temp['url'] = i
game_id_df_temp = game_id_df_temp.drop(columns = ['index'])

game_id_df = game_id_df.append(game_id_df_temp).reset_index()
game_id_df = game_id_df.drop(columns = ['index'])

In [49]:
print(game_id_df.shape)
game_id_df.tail()

(345, 2)


,id,url
340,401437928,https://www.espn.com/nfl/scoreboard/_/week/16/...
341,401437929,https://www.espn.com/nfl/scoreboard/_/week/16/...
342,401436983,https://www.espn.com/nfl/scoreboard/_/week/16/...
343,401437930,https://www.espn.com/nfl/scoreboard/_/week/16/...
344,401437931,https://www.espn.com/nfl/scoreboard/_/week/16/...


In [50]:
game_id_df.to_csv('data/game_id_catch_up.csv')

## Scrape Game Odds

In [316]:
## hitting this api will show the current weeks games and the betting data behind each game

#driver.get('https://www.espn.com/nfl/scoreboard')

url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America/New_York"

payload={}
headers = {
  'Accept': 'application/json',
  'Cookie': 'SWID=3750444F-BA06-4929-C99C-62333D7EE8A0'
}

response = requests.request("GET", url, headers=headers, data=payload)

response_text = json.loads(response.text)

game_df = json_normalize(response_text['sports'], ['leagues', 'events'])

## include this data after predictions to find valuable bets that show some different towhat vegas thinks

In [317]:
game_df.head()

,gamecastAvailable,playByPlayAvailable,commentaryAvailable,recent,id,competitionId,uid,date,timeValid,name,...,odds.homeTeamOdds.moneyLine,odds.homeTeamOdds.spreadOdds,odds.homeTeamOdds.team.id,odds.homeTeamOdds.team.abbreviation,odds.links,odds.moneylineWinner,odds.spreadWinner,odds.home.moneyLine,odds.away.moneyLine,fullStatus.type.altDetail
0,False,False,False,False,401326497,401326497,s:20~l:28~e:401326497~c:401326497,2021-11-28T18:00:00Z,True,Pittsburgh Steelers at Cincinnati Bengals,...,-180,-110.0,4,CIN,[],False,False,-180,155,NaN
1,False,False,False,False,401326499,401326499,s:20~l:28~e:401326499~c:401326499,2021-11-28T18:00:00Z,True,Tampa Bay Buccaneers at Indianapolis Colts,...,130,100.0,11,IND,[],False,False,130,-150,NaN
2,False,False,False,False,401326501,401326501,s:20~l:28~e:401326501~c:401326501,2021-11-28T18:00:00Z,True,Carolina Panthers at Miami Dolphins,...,100,-115.0,15,MIA,[],False,False,100,-120,NaN
3,False,False,False,False,401326502,401326502,s:20~l:28~e:401326502~c:401326502,2021-11-28T18:00:00Z,True,Tennessee Titans at New England Patriots,...,-340,-105.0,17,NE,[],False,False,-340,270,NaN
4,False,False,False,False,401326503,401326503,s:20~l:28~e:401326503~c:401326503,2021-11-28T18:00:00Z,True,Philadelphia Eagles at New York Giants,...,175,-110.0,19,NYG,[],False,False,175,-210,NaN


## scraping individual game data -- we just need to find a way we get scrap all the game id's and then pass them to the functions below

In [51]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [57]:
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.espn.com/nfl/matchup?gameId=401326425').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
#pprint(p.tables[1])

df_testing = pd.DataFrame(p.tables[1])
column_list = list(df_testing[0].values)

away = pd.DataFrame(df_testing[1]).T
away.columns = column_list
away = away.reset_index()
# away = away.drop(columns = ['index', 'Matchup'])
away = away.drop(columns = ['index'])
#away.drop(away.columns[[13]], axis=1, inplace=True)
away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

home = pd.DataFrame(df_testing[2]).T
home.columns = column_list
home = home.reset_index()
# home = home.drop(columns = ['index', 'Matchup'])
home = home.drop(columns = ['index'])
#home.drop(home.columns[[13]], axis=1, inplace=True)
home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

game_stat_temp = pd.concat([away, home], axis=1)
game_stat_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Team Stats                  1 non-null      object
 1   first_downs_away            1 non-null      object
 2   first_downs_passing_away    1 non-null      object
 3   first_down_rushing_away     1 non-null      object
 4   first_down_by_penalty_away  1 non-null      object
 5   third_down_away             1 non-null      object
 6   fourth_down_away            1 non-null      object
 7   total_plays_away            1 non-null      object
 8   total_yards_away            1 non-null      object
 9   total_drives_away           1 non-null      object
 10  yards_per_play_away         1 non-null      object
 11  passing_away                1 non-null      object
 12  comp_att_away               1 non-null      object
 13  yards_per_pass_away         1 non-null      object
 14

In [58]:
print('Level Two gets you each row...', p.tables[1][1])
print('row header...', p.tables[1][1][0])
print('away data point...', p.tables[1][1][1])
print('home data point...', p.tables[1][1][2])

Level Two gets you each row... ['1st Downs', '25', '19']
row header... 1st Downs
away data point... 25
home data point... 19


### need to just write a loop to go through this table and extract all points

In [59]:
game_stats_big_df = pd.DataFrame()

for i in game_id_df['id']:
    
    try:
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        # defining the html contents of a URL.
        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        # Now finally obtaining the data of
        # the table required
        df_testing = pd.DataFrame(p.tables[1])
        column_list = list(df_testing[0].values)

        away = pd.DataFrame(df_testing[1]).T
        away.columns = column_list
        away = away.reset_index()
        # away = away.drop(columns = ['index', 'Matchup'])
        away = away.drop(columns = ['index'])
        #away.drop(away.columns[[13]], axis=1, inplace=True)
        away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',
                            'Fumbles Lost': 'fumbles_lost_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

        home = pd.DataFrame(df_testing[2]).T
        home.columns = column_list
        home = home.reset_index()
        # home = home.drop(columns = ['index', 'Matchup'])
        home = home.drop(columns = ['index'])
        #home.drop(home.columns[[13]], axis=1, inplace=True)
        home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',
                            'Fumbles Lost': 'fumbles_lost_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

        game_stat_temp = pd.concat([away, home], axis=1)
        game_stat_temp['id'] = i
        game_stats_big_df = game_stats_big_df.append(game_stat_temp)
        game_stats_big_df = game_stats_big_df.drop(columns = ['Team Stats'])

    except Exception as e:
        print("Error:", e)
    

In [62]:
game_stats_big_df = game_stats_big_df.drop(columns = ['Team Stats'])
game_stats_big_df.shape

(345, 51)

In [61]:
game_stats_big_df.columns

Index(['Team Stats', 'first_downs_away', 'first_downs_passing_away',
       'first_down_rushing_away', 'first_down_by_penalty_away',
       'third_down_away', 'fourth_down_away', 'total_plays_away',
       'total_yards_away', 'total_drives_away', 'yards_per_play_away',
       'passing_away', 'comp_att_away', 'yards_per_pass_away',
       'int_thrown_away', 'sack_yards_lost_away', 'rushing_away',
       'rushing_att_away', 'yards_per_rush_away', 'red_zone_att_away',
       'penalty_away', 'turnovers_away', 'Fumbles lost', 'int_thrown_away',
       'defensive_td_away', 'possession_away', 'Team Stats',
       'first_downs_home', 'first_downs_passing_home',
       'first_down_rushing_home', 'first_down_by_penalty_home',
       'third_down_home', 'fourth_down_home', 'total_plays_home',
       'total_yards_home', 'total_drives_home', 'yards_per_play_home',
       'passing_home', 'comp_att_home', 'yards_per_pass_home',
       'int_thrown_home', 'sack_yards_lost_home', 'rushing_home',
       '

In [63]:
game_stats_big_df.to_csv('data/game_stats_catch_up.csv')


In [294]:
game_score_big_df = pd.DataFrame()

for i in game_id_df['id']:
    
    try:
        # defining the html contents of a URL.
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        df_score = pd.DataFrame(p.tables[0])
        df_score = pd.DataFrame(df_score.iloc[1: , :])
        team_column = df_score[0].to_frame()
        #team_column = team_column.rename(columns = {0: 'team'})
        #team_df = pd.DataFrame(team_column, columns = ['team'])
        score_column = df_score.iloc[: , -1].to_frame()
        #score_column = score_column.rename({: 'score'})
        #score_df = pd.DataFrame(score_column, columns = ['score'])
        df_score_new = pd.concat([team_column, score_column], axis=1)
        df_score_new.columns = ['team', 'score']

        away_score = pd.DataFrame(df_score_new.iloc[0]).T
        away_score = away_score.reset_index()
        away_score = away_score.drop(columns = ['index'])
        #away_score = away_score[['team', 'score']]
        away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

        home_score = pd.DataFrame(df_score_new.iloc[1]).T
        home_score = home_score.reset_index()
        home_score = home_score.drop(columns = ['index'])
        #home_score = home_score[['team', 'score']]
        home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

        game_score_temp = pd.concat([away_score, home_score], axis=1)
        game_score_temp['id'] = i
        #game_score_temp = game_score_temp.reset_index()
        #game_score_temp = game_score_temp.drop(columns = ['index'])
        game_score_big_df = game_score_big_df.append(game_score_temp)

    except Exception as e:
        print('Error:', e, 'game_id=', i)

In [295]:
game_score_big_df

,away_team,away_score,home_team,home_score,id
0,BAL,10,MIA,22,401326465
0,ATL,3,DAL,43,401326466
0,NO,21,TEN,23,401326471
0,JAX,17,IND,23,401326467
0,CLE,7,NE,45,401326468
0,BUF,45,NYJ,17,401326469
0,DET,16,PIT,16,401326470
0,TB,19,WSH,29,401326472
0,CAR,34,ARI,10,401326473
0,MIN,27,LAC,20,401326474


In [296]:
game_score_big_df.to_csv('data/game_scores_catch_up.csv')

## appendix - just needed to get everything together from seperate files

In [297]:
game_id_og = pd.read_csv('data/game_id_all.csv')
game_id_new = pd.read_csv('data/game_id_catch_up.csv')
game_id_all = game_id_og.append(game_id_new)
print(game_id_all.shape)

game_scores_og = pd.read_csv('data/game_scores.csv')
game_scores_new = pd.read_csv('data/game_scores_catch_up.csv')
game_scores_all = game_scores_og.append(game_scores_new)
print(game_scores_all.shape)

game_stats_og = pd.read_csv('data/game_stats_all.csv')
game_stats_new = pd.read_csv('data/game_stats_catch_up.csv')
game_stats_all = game_stats_og.append(game_stats_new)
print(game_stats_all.shape)

(5166, 3)
(5043, 6)
(5028, 52)


In [314]:
game_id_all = game_id_all.drop(columns = 'Unnamed: 0')
game_id_all.tail()

,id,url
24,401326489,https://www.espn.com/nfl/scoreboard/_/week/11/...
25,401326490,https://www.espn.com/nfl/scoreboard/_/week/11/...
26,401326491,https://www.espn.com/nfl/scoreboard/_/week/11/...
27,401326492,https://www.espn.com/nfl/scoreboard/_/week/11/...
28,401326493,https://www.espn.com/nfl/scoreboard/_/week/11/...


In [315]:
game_id_all.to_csv('data/game_id_all.csv')

In [301]:
game_stats_all = game_stats_all.drop(columns = 'Unnamed: 0')
game_stats_all.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,rushing_att_home,yards_per_rush_home,red_zone_att_home,penalty_home,turnovers_home,Fumbles lost.1,int_thrown_home.1,defensive_td_home,possession_home,id
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,22,2.0,0-6,10-80,3,0,3,0,32:28,220905019
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,32,4.4,0-8,10-82,3,1,2,0,39:08,220908002
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,26,3.1,0-6,4-33,2,1,1,0,28:29,220908003
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,13,2.8,0-5,9-57,1,0,1,0,22:12,220908004
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,20,3.0,0-10,4-38,1,1,0,0,29:44,220908005


In [303]:
game_scores_all = game_scores_all.drop(columns = 'Unnamed: 0')
game_scores_all.head()

,away_team,away_score,home_team,home_score,id
0,SF,16,NYG,13,220905019
1,NYJ,37,BUF,31,220908002
2,MIN,23,CHI,27,220908003
3,SD,34,CIN,6,220908004
4,KC,40,CLE,39,220908005


In [304]:
game_details_combined = game_stats_all.merge(game_scores_all, on='id', how='inner')
print(game_details_combined.shape)
game_details_combined

(5053, 55)


,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,turnovers_home,Fumbles lost.1,int_thrown_home.1,defensive_td_home,possession_home,id,away_team,away_score,home_team,home_score
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,3,0,3,0,32:28,220905019,SF,16,NYG,13
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,3,1,2,0,39:08,220908002,NYJ,37,BUF,31
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,2,1,1,0,28:29,220908003,MIN,23,CHI,27
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,1,0,1,0,22:12,220908004,SD,34,CIN,6
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,1,1,0,0,29:44,220908005,KC,40,CLE,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5048,23,10,10,3,8-16,0-0,70,288,10,4.1,...,2,1,1,0,22:40,401326489,CIN,32,LV,13
5049,16,14,1,1,5-15,0-0,64,276,12,4.3,...,2,1,1,0,30:47,401326490,DAL,9,KC,19
5050,29,21,6,2,7-14,1-1,79,413,10,5.2,...,0,0,0,0,19:38,401326491,ARI,23,SEA,13
5051,22,14,5,3,8-15,0-2,65,300,11,4.6,...,1,0,1,0,31:46,401326492,PIT,37,LAC,41


In [305]:
game_details_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5053 entries, 0 to 5052
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   first_downs_away            5053 non-null   int64  
 1   first_downs_passing_away    5053 non-null   int64  
 2   first_down_rushing_away     5053 non-null   int64  
 3   first_down_by_penalty_away  5053 non-null   int64  
 4   third_down_away             5053 non-null   object 
 5   fourth_down_away            5053 non-null   object 
 6   total_plays_away            5053 non-null   int64  
 7   total_yards_away            5053 non-null   int64  
 8   total_drives_away           5053 non-null   int64  
 9   yards_per_play_away         5053 non-null   float64
 10  passing_away                5053 non-null   int64  
 11  comp_att_away               5053 non-null   object 
 12  yards_per_pass_away         5053 non-null   float64
 13  int_thrown_away             5053 

In [308]:
game_details_combined = game_details_combined.rename(columns={'Fumbles lost': 'fumbles_lost_away', 'Fumbles lost.1': 'fumbles_lost_home'})
game_details_combined = game_details_combined.drop(columns=['int_thrown_home.1', 'int_thrown_away.1'])
game_details_combined.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,penalty_home,turnovers_home,fumbles_lost_home,defensive_td_home,possession_home,id,away_team,away_score,home_team,home_score
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,10-80,3,0,0,32:28,220905019,SF,16,NYG,13
1,18,11,4,3,2-8,0-0,47,266,17,5.7,...,10-82,3,1,0,39:08,220908002,NYJ,37,BUF,31
2,19,9,9,1,5-13,0-0,62,368,17,5.9,...,4-33,2,1,0,28:29,220908003,MIN,23,CHI,27
3,27,9,16,2,6-10,0-0,65,401,11,6.2,...,9-57,1,0,0,22:12,220908004,SD,34,CIN,6
4,24,12,10,2,5-11,0-0,59,470,20,8.0,...,4-38,1,1,0,29:44,220908005,KC,40,CLE,39


In [310]:
game_details_combined.to_csv('data/nfl_historical_game_detail.csv')